# sample generator for embedded questions
This notebook is for creating the random sample .txt files for the experiment.

In [61]:
import pandas as pd
import numpy as np

 # Contents
 1. [Constrain the data set to stimuli set](#Constrain-the-dataset-to-stimuli-set)
 2. [Figuring out the distribution of factors per list](#Figuring-out-the-distribution-of-factors-per-list)
 3. [Figure out how to collapse the matrix verb columns](#Figure-out-how-to-collapse-the-matrix-verb-columns)
 4. [Add in the paraphrases](#Add-in-the-paraphrases)
 5. [Split EntireSentence on Question](#Split-EntireSentence-on-Question)
 6. [Controls](#Controls)
 7. [Balancing factors](#Balancing-factors)
     1. [Modal Balancing](#Modal-Balancing)
     2. [Wh Balancing](#Wh-Balancing)
         1. [Who](#Who)
         2. [What](#What)
         3. [Where](#Where)
         4. [When](#When)
         5. [How](#How)
         6. [Why](#Why)
 8. [Generating-random-samples](#Generating-random-samples)
     1. [First Iteration](#First-Iteration)
     2. [Second Iteration](#Second-Itreation)
     3. [Third Iteration](#Third-Iteration)
     4. [Fourth Iteration](#Fourth-Iteration)
     5. [Fifth Iteration](#Fifth-Iteration)
     6. [Sixth Iteration](#Sixth-Iteration)
     7. [Final Set](#Final-Set)
 9. [Pilot Samples](#Pilot-Samples)

In [62]:
# import the database file from the TGrep2 searching
df = pd.read_csv("../results/swbd.tab", sep='\t', engine='python')

In [63]:
df.head()

,Item_ID,Sentence,HaveNeedTo,Finite,ModalPresent,QuestionType,EmbeddedSQ,DegreeQ,SubjectAuxInv,WhAll,...,FullWhPhrase,JustMatrixClause,DeterminerSubjPresent,DeterminerNonSubjPresent,WhNode,WhParse,Question,SentenceParse,WhPhaseType,IdentityQ
0,3:43,"uh, first, um, i need *-1 to know, uh, how do you feel *t*-2 about, uh, about * sending, uh, an elderly, uh, family member to a nursing home?",no,yes,no,embedded,yes,no,yes,NaN,...,NaN,"*-1 to know, uh, how do you feel *t*-2 about, uh, about * sending, uh, an elderly, uh, family member to a nursing home",no,no,NaN,(WRB how),"how do you feel *t*-2 about, uh, about * sending, uh, an elderly, uh, family member to a nursing home","(TOP (S (INTJ (UH Uh)) (, ,) (ADVP-TMP (RB first)) (, ,) (INTJ (UH um)) (, ,) (NP-SBJ-1 (PRP I)) (VP (VBP need) (S (NP-SBJ (-NONE- *-1)) (VP (TO to) (VP (VB know) (, ,) (INTJ (UH uh)) (, ,) (SBARQ (WHADVP-2 (WRB how)) (SQ (VBP do) (NP-SBJ (PRP you)) (VP (VB feel) (ADVP (-NONE- *T*-2)) (EDITED (RM (-DFL- \[)) (PP-UNF (IN about)) (, ,) (IP (-DFL- \+))) (INTJ (UH uh)) (, ,) (PP (IN about) (RS (-DFL- \])) (S-NOM (NP-SBJ (-NONE- *)) (VP (VBG sending) (, ,) (INTJ (UH uh)) (, ,) (NP (DT an) (JJ elderly) (, ,) (INTJ (UH uh)) (, ,) (NN family) (NN member)) (PP-DIR (IN to) (NP (DT a) (NN nursing) (NN home))))))))))))) (. ?) (-DFL- E_S)))",monomorphemic,NaN
1,17:77,"and, uh, we were, i was fortunate in that i was personally acquainted with the, uh, people who, uh, *t*-1 ran the nursing home in our little hometown.",no,yes,no,subject,NaN,no,NaN,NaN,...,who,NaN,no,no,NaN,(WP who),"who, uh, *t*-1 ran the nursing home in our little hometown","(TOP (S (CC And) (, ,) (INTJ (UH uh)) (, ,) (EDITED (RM (-DFL- \[)) (S (NP-SBJ (PRP we)) (VP-UNF (VBD were))) (, ,) (IP (-DFL- \+))) (NP-SBJ (PRP I)) (VP (VBD was) (RS (-DFL- \])) (ADJP-PRD (JJ fortunate) (SBAR-PRP (IN in) (IN that) (-DFL- E_S) (S (NP-SBJ (PRP I)) (VP (VBD was) (ADJP-PRD (RB personally) (VBN acquainted) (PP (IN with) (NP (NP (DT the) (, ,) (INTJ (UH uh)) (, ,) (NNS people)) (SBAR (WHNP-1 (WP who)) (, ,) (INTJ (UH uh)) (, ,) (S (NP-SBJ (-NONE- *T*-1)) (VP (VBD ran) (NP (NP (DT the) (NN nursing) (NN home)) (PP-LOC (IN in) (NP (PRP$ our) (JJ little) (NN hometown))))))))))))))) (. .) (-DFL- E_S)))",monomorphemic,no
2,21:45,"so, i was very comfortable, you know, in *-1 doing it when it got to the point that we had *-2 to do it *t*-3 *t*-4.",yes,yes,no,embadjunct,NaN,no,NaN,NaN,...,when,*-1 doing it when it got to the point that we had *-2 to do it *t*-3 *t*-4,no,no,NaN,(WRB when),when it got to the point that we had *-2 to do it *t*-3 *t*-4,"(TOP (S (RB So) (, ,) (NP-SBJ-1 (PRP I)) (VP (VBD was) (ADJP-PRD (RB very) (JJ comfortable) (PRN (, ,) (S (NP-SBJ (PRP you)) (VP (VBP know))) (, ,)) (PP (IN in) (S-NOM (NP-SBJ (-NONE- *-1)) (VP (VBG doing) (NP (PRP it)) (SBAR-TMP (WHADVP-4 (WRB when)) (S (NP-SBJ (PRP it)) (VP (VBD got) (PP (IN to) (NP (NP (DT the) (NN point)) (SBAR (WHADVP-3 (WDT that)) (S (NP-SBJ-2 (PRP we)) (VP (VBD had) (S (NP-SBJ (-NONE- *-2)) (VP (TO to) (VP (VB do) (NP (PRP it)) (ADVP-TMP (-NONE- *T*-3)))))))))) (ADVP-TMP (-NONE- *T*-4)))))))))) (. .) (-DFL- E_S)))",monomorphemic,no
3,23:31,"well, i had an occasion for my mother-in-law who *t*-1 had fell and needed * to be, you know, could not take care of herself anymore, was confined *-2 to a nursing home for a while",yes,NaN,yes,relative,NaN,no,NaN,NaN,...,who,NaN,no,no,NaN,(WP who),"who *t*-1 had fell and needed * to be, you know, could not take care of herself anymore, was confined *-2 to a nursing home for a while","(TOP (S (INTJ (UH well)) (, ,) (NP-SBJ (PRP I)) (VP (VBD had) (NP (NP (DT an) (NN occasion)) (PP (IN for) (NP (NP (PRP$ my) (NN mother-in-law)) (SBAR (WHNP-1 (WP who)) (S (NP-SBJ-2 (-NONE- *T*-1)) (VP (VP (VBD had) (VP (VBN fell))) (CC and) (EDITED (RM (-DFL- \[)) (VP (VBD needed) (S (NP-SBJ (-NONE- *)) (VP (TO to) (VP-UNF (VB be))))) (, ,) (IP (-DFL- \+)) (RS (-DFL- \]))) (PRN (S (NP-SBJ (PRP you)) (VP (VBP know))) (, ,)) (VP (MD could) (RB not)

In [64]:
# This makes the display show more info
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [65]:
df.pivot_table(index=['QuestionType'], values="Question", aggfunc=len).groupby(["QuestionType"]).Question.transform(lambda x: x/len(df)).reset_index()

,QuestionType,Question
0,adjunct,0.075203
1,cleft,0.064712
2,embadjunct,0.236886
3,embedded,0.160800
4,fragment,0.067654
5,relative,0.133150
6,root,0.124424
7,subject,0.137072


# Constrain the dataset to stimuli set
for experimental mock-up

First we have to remove the questions that we don;t want to include:
1. embedded questions only
2. no degree questions
3. no identity questions
4. generally only monomorphemic wh-phrases
5. only who-, what-, where-, when-, how-, and why-questions

In [66]:
critical = df[(df['QuestionType'] == 'embedded') # only embedded questions
              & 
              (df['Finite'] == 'no' )] # infinitival clauses
            

In [67]:
len(critical)

135

In [68]:
7*30

210

### Figuring out how many lists

135/20

6 lists of 20, 1 of 15

15 how / 6list
4 whats / 6 lists
then random sample 1 item from the remainders

last list is whatever's left over (total 15 items)


### Number of participants
7Lists*30 subjects/list = 210

# Combine contexts with constrained db

In [69]:
# read in df with contexts
cntxts = pd.read_csv("swbd_contexts.csv")

In [70]:
cntxts = cntxts.drop(columns="FollowingContext")

In [71]:
# get the indixes from critical
crit_index = critical.Item_ID

### Merge back in Wh colum
so we can sample proportionately based on Wh

In [72]:
df_Wh = critical[["Item_ID","Wh","Question"]].rename(columns={"Item_ID": "TGrepID"})

In [73]:
# subset to the items that are just the ones filtered in the previos section

# otherwise, if using the database file with contexts directly in there, then this step
# is not necessary
df_valid = cntxts[cntxts["TGrepID"].isin(set(crit_index))]

In [74]:
# Merge
df_valid = df_valid.merge(df_Wh, how = 'inner', indicator=False)

In [75]:
df_valid.head()

,TGrepID,EntireSentence,PreceedingContext,Wh,Question
0,1049:139,"i also thought about it, was of, uh, *-1 waiting *-2 to talk to you that, another thing that *t*-4 occurred to me is 0 there is not so much invasion of my privacy because i know how *-5 to behave *t*-6 such that there isn't *?*.","###speakera67.###yeah.###speakerb68.###so, maybe that is a, a little bit of what privacy is *t*-1.###speakera69.###yes.###exactly.###speakerb70.###speakera71.###uh-huh.",how,how *-5 to behave *t*-6 such that there isn't *?*
1,1130:56,"and if that's gone, um, i, i really don't know how *-1 to live *t*-2 very well,","###i actually###for al-, of the time 0 i've spent *t*-1 there, i still don't quite understand how certain things that i assume 0 *t*-2 and require privacy and require not just that you be alone but actually that you have a sense of privacy *t*-3.###speakerb94.###yes.###speakera95.###because anyone can be alone for some period of time###but for me a lot of what i do *t*-1 requires a sense that there's this invisible barrier around me which people wil-, will respect *t*-2.###speakerb96.###yes.###speakera97.",how,how *-1 to live *t*-2 very well
2,4531:36,"and more and more people start *-2 believing them or wondering how * to combine them with other things *t*-1,","###and, part of it is 0 california, you know, in, back in the sixties, had a lot of alternative movements###speakerb98.###speakera99.###and some of them fizzled out###and some of them were disastrous###and others of them, um, had an impact on the society around here.###and one of the ones that *t*-1 had an impact was, uh, people becoming interested in alternate practices,###i'm not sure if it was a meditation practice, or if it was, you know, which *t*-1 is similar to a stress management practice or alternates to, uh, a m a approved medicine.###uh, you have, you know, major, um, acupuncture schools and things out here.### and, and you could have them around long enough",how,how * to combine them with other things *t*-1
3,6397:23,"so, i really don't know how * to compare it to other big companies *t*-1, you know.","###speakera67.###speakerb68.###that's right.###well, you know, t i, you know, t i offers some good stuff###and then i think 0 there's, i mean i think 0 there's some negatives,###but there's going *-1 to be some negatives anywhere, you know, no matter where you go *t*-2.###i have, you know, all,###this is the first really large company 0 i've worked for *t*-1.###i've always been involved in little small, you know, ind-, privately owned s-, owned firms###and so i've never had the, the big benefit package.",how,how * to compare it to other big companies *t*-1
4,7981:19,"you don't even know who *-1 to payoff *t*-2, huh?.","###speakerb90.###yeah,###speakera91.###most of the time.###speakerb92.###but the politics, the politics gets worse in the small towns sometimes.###speakera93.###oh man, in dallas you don't even know who *t*-1's in, in administration,###there's so many of them.###speakerb94.",who,who *-1 to payoff *t*-2


# Split EntireSentence on Question 
This is necessary because we need to bold the question only and not the Matrix in the experimental file

In [76]:
# split EntireSentence at the string that matches the value in the 'question' column
df_valid["Matrix"] = df_valid.apply(lambda x: x['EntireSentence'].replace(x['Question'],"").strip(),axis=1)

In [77]:
# split that last punctuation off, to be added back on in .js script
df_valid["punct"] = df_valid["Matrix"].apply(lambda x: list(x)[-1])

In [78]:
# remove that final punct from the Matrix column
df_valid["Matrix"] = df_valid["Matrix"].apply(lambda x: x.replace(list(x)[-1], ' '))

In [79]:
df_valid.head()

,TGrepID,EntireSentence,PreceedingContext,Wh,Question,Matrix,punct
0,1049:139,"i also thought about it, was of, uh, *-1 waiting *-2 to talk to you that, another thing that *t*-4 occurred to me is 0 there is not so much invasion of my privacy because i know how *-5 to behave *t*-6 such that there isn't *?*.","###speakera67.###yeah.###speakerb68.###so, maybe that is a, a little bit of what privacy is *t*-1.###speakera69.###yes.###exactly.###speakerb70.###speakera71.###uh-huh.",how,how *-5 to behave *t*-6 such that there isn't *?*,"i also thought about it, was of, uh, *-1 waiting *-2 to talk to you that, another thing that *t*-4 occurred to me is 0 there is not so much invasion of my privacy because i know",.
1,1130:56,"and if that's gone, um, i, i really don't know how *-1 to live *t*-2 very well,","###i actually###for al-, of the time 0 i've spent *t*-1 there, i still don't quite understand how certain things that i assume 0 *t*-2 and require privacy and require not just that you be alone but actually that you have a sense of privacy *t*-3.###speakerb94.###yes.###speakera95.###because anyone can be alone for some period of time###but for me a lot of what i do *t*-1 requires a sense that there's this invisible barrier around me which people wil-, will respect *t*-2.###speakerb96.###yes.###speakera97.",how,how *-1 to live *t*-2 very well,and if that's gone um i i really don't know,","
2,4531:36,"and more and more people start *-2 believing them or wondering how * to combine them with other things *t*-1,","###and, part of it is 0 california, you know, in, back in the sixties, had a lot of alternative movements###speakerb98.###speakera99.###and some of them fizzled out###and some of them were disastrous###and others of them, um, had an impact on the society around here.###and one of the ones that *t*-1 had an impact was, uh, people becoming interested in alternate practices,###i'm not sure if it was a meditation practice, or if it was, you know, which *t*-1 is similar to a stress management practice or alternates to, uh, a m a approved medicine.###uh, you have, you know, major, um, acupuncture schools and things out here.### and, and you could have them around long enough",how,how * to combine them with other things *t*-1,and more and more people start *-2 believing them or wondering,","
3,6397:23,"so, i really don't know how * to compare it to other big companies *t*-1, you know.","###speakera67.###speakerb68.###that's right.###well, you know, t i, you know, t i offers some good stuff###and then i think 0 there's, i mean i think 0 there's some negatives,###but there's going *-1 to be some negatives anywhere, you know, no matter where you go *t*-2.###i have, you know, all,###this is the first really large company 0 i've worked for *t*-1.###i've always been involved in little small, you know, ind-, privately owned s-, owned firms###and so i've never had the, the big benefit package.",how,how * to compare it to other big companies *t*-1,"so, i really don't know , you know",.
4,7981:19,"you don't even know who *-1 to payoff *t*-2, huh?.","###speakerb90.###yeah,###speakera91.###most of the time.###speakerb92.###but the politics, the politics gets worse in the small towns sometimes.###speakera93.###oh man, in dallas you don't even know who *t*-1's in, in administration,###there's so many of them.###speakerb94.",who,who *-1 to payoff *t*-2,"you don't even know , huh?",.


# Generating random samples

## First Iteration

6 lists of 20, 1 of 15

15 how / 6 list
4 whats / 6 lists
then random sample 1 item from the remainders

last list is whatever's left over (total 15 items)

In [80]:
for n in range(1,7):
    how_sample = df_valid[df_valid["Wh"] == "how"].sample(15)
    df_valid = df_valid.drop(how_sample.index)    
    
    what_sample = df_valid[df_valid["Wh"] == "what"].sample(4)
    df_valid = df_valid.drop(what_sample.index)
    
    final_sample = df_valid[
        (df_valid["Wh"] != "how") &
        (df_valid["Wh"] != "what")
    ].sample(1)
    df_valid = df_valid.drop(final_sample.index)

    
    total = pd.concat([how_sample,what_sample,final_sample])
    # save to file
    filename = f"../../experiments/clean_corpus/01_experiment/corpus_{n}.txt".format(n=n)
    total.to_csv(filename,header=True,sep="\t",index=False)
    
    
    
    

## Final Set

List = 15

10 how

In [81]:
len(df_valid)

15